In [26]:
import emoji

import mlflow
from mlflow.models import infer_signature
from mlflow import MlflowClient

import sklearn.datasets
from sklearn.metrics import accuracy_score
from sklearn.model_selection import train_test_split

import xgboost as xgb

import ray
from ray import train, tune
from ray.air.integrations.mlflow import setup_mlflow

TRACKING_URI = "http://0.0.0.0:5000"
EXP_NAME = "ml_platform"
MODEL_NAME = "xgb-breast-cancer-classifer"
ALIAS = "champion"
MINIMUM_REQUIRED_ACCURACY = 0.95

mlflow.set_tracking_uri(TRACKING_URI)
mlflow.set_experiment(experiment_name=EXP_NAME)

client = MlflowClient()

In [3]:
def train_breast_cancer(config: dict) -> None:
    setup_mlflow(
        config,
        experiment_name=EXP_NAME,
        tracking_uri=TRACKING_URI,
    )
    # Load dataset
    data, labels = sklearn.datasets.load_breast_cancer(return_X_y=True)
    train_x, test_x, train_y, test_y = train_test_split(data, labels, test_size=0.2)

    model = xgb.XGBClassifier(**config)
    model.fit(train_x, train_y)

    predictions = model.predict(test_x)
    accuracy = accuracy_score(test_y, predictions)
    signature = infer_signature(train_x, model.predict(train_x))

    mlflow.log_metric("accuracy", accuracy)
    mlflow.xgboost.log_model(
        model,
        "xgb_models",
        conda_env=mlflow.xgboost.get_default_conda_env(),
        signature=signature,
        model_format="json",
    )

    train.report({"accuracy": accuracy, "done": True})


def tune_with_setup() -> None:

    ray.init(num_cpus=6)
    trainable_with_resources = tune.with_resources(train_breast_cancer, {"cpu": 2})

    tuner = tune.Tuner(
        trainable_with_resources,
        tune_config=tune.TuneConfig(
            num_samples=20,
        ),
        run_config=train.RunConfig(
            name="mlflow",
        ),
        param_space={
            "objective": "binary:logistic",
            "eval_metric": ["logloss", "error"],
            "max_depth": tune.randint(1, 9),
            "min_child_weight": tune.choice([1, 2, 3]),
            "subsample": tune.uniform(0.5, 1.0),
            "eta": tune.loguniform(1e-4, 1e-1),
        },
    )

    tuner.fit()

In [4]:
tune_with_setup()

2025-01-27 23:48:09,428	INFO tune.py:1009 -- Wrote the latest version of all result files and experiment state to '/home/ssafarveisi/ray_results/mlflow' in 0.0043s.
2025-01-27 23:48:09,435	INFO tune.py:1041 -- Total run time: 31.56 seconds (31.42 seconds for the tuning loop).


In [5]:
from mlflow.entities import ViewType

experiment_id = mlflow.get_experiment_by_name(name=EXP_NAME).experiment_id
runs = mlflow.search_runs(
    experiment_ids=[experiment_id], run_view_type=ViewType.ACTIVE_ONLY
)

In [13]:
best_run = runs.loc[runs["metrics.accuracy"].idxmax()]
best_run_id = best_run.run_id
best_run_accuracy = best_run["metrics.accuracy"]
model_uri = f"runs:/{best_run_id}/xgb_models"

In [34]:
if best_run_accuracy > MINIMUM_REQUIRED_ACCURACY:
    print(emoji.emojize("Model accuracy met the required minimum accuracy :fire:"))
    result = mlflow.register_model(model_uri, MODEL_NAME)
else:
    print(
        emoji.emojize(
            "Best run did not meet the required minimum accuracy :sad_but_relieved_face:"
        )
    )

Registered model 'xgb-breast-cancer-classifer' already exists. Creating a new version of this model...
2025/01/28 00:07:06 INFO mlflow.store.model_registry.abstract_store: Waiting up to 300 seconds for model version to finish creation. Model name: xgb-breast-cancer-classifer, version 6


Model accuracy met the required minimum accuracy 🔥


Created version '6' of model 'xgb-breast-cancer-classifer'.


In [28]:
client.set_registered_model_alias(MODEL_NAME, ALIAS, result.version)

In [29]:
champion_version = mlflow.pyfunc.load_model(f"models:/{MODEL_NAME}@{ALIAS}")

In [30]:
data, labels = sklearn.datasets.load_breast_cancer(return_X_y=True)
_, test_x, _, test_y = train_test_split(data, labels, test_size=0.2)


champion_version.predict(test_x)

array([1, 1, 1, 1, 0, 1, 1, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0, 1, 1, 0, 1, 1,
       1, 0, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 0, 1, 1, 1, 1, 0, 0, 1, 0, 0,
       0, 1, 0, 1, 1, 0, 1, 1, 1, 1, 1, 0, 1, 0, 1, 1, 1, 1, 0, 0, 1, 1,
       1, 0, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 0, 1, 0, 1, 1, 1, 1, 0, 1,
       0, 0, 1, 1, 1, 1, 0, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 0, 0, 1, 1,
       1, 0, 1, 0])